In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

In [2]:
train = pd.read_csv("train_v9rqX0R.csv")
test = pd.read_csv("test_AbJTz2l.csv")

In [3]:
train.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [5]:
train.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [6]:
train = train.fillna(train.median())

train = train.fillna("NA")

In [7]:
X = train.copy()
X = X.drop(['Item_Identifier','Item_Outlet_Sales'],axis=1)
y = train['Item_Outlet_Sales']

In [8]:
test = test.fillna(train.median())

test = test.fillna("NA")

In [9]:
X_test = test.copy()
X_test = X_test.drop(['Item_Identifier'],axis=1)

In [10]:
result = pd.concat([X, X_test])

In [11]:
result.shape

(14204, 10)

In [12]:
result = pd.get_dummies(result,drop_first=True)

In [13]:
X = result[:8523]
X_test = result[8523:]

In [14]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [15]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=1)

In [16]:
X_train.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Fat_Content_low fat,Item_Fat_Content_reg,Item_Type_Breads,Item_Type_Breakfast,...,Outlet_Identifier_OUT046,Outlet_Identifier_OUT049,Outlet_Size_Medium,Outlet_Size_NA,Outlet_Size_Small,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
1945,18.35,0.089345,191.9504,2009,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
1720,17.35,0.168065,176.2712,2009,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
1954,10.10,0.053887,225.6088,2007,0,1,0,0,0,0,...,0,0,0,1,0,1,0,1,0,0
1919,10.85,0.162904,104.9622,2009,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
2461,7.17,0.059717,130.9968,2004,0,1,0,0,0,0,...,0,0,0,0,1,1,0,1,0,0


In [17]:
from sklearn.model_selection import cross_val_score

# Linear Regression

In [18]:
from sklearn.linear_model import LinearRegression

In [19]:
lr = LinearRegression()
# lr.fit(X_train, y_train)
# lr.score(X_valid, y_valid)

In [20]:
# y_pred = lr.predict(y_valid)

In [21]:
from sklearn.metrics import mean_squared_error

In [23]:
# mean_squared_error(y_valid, y_pred, squared=False)

In [24]:
scores = cross_val_score(lr, X, y, cv=3,scoring='neg_root_mean_squared_error')
np.mean(scores)

-1133.7405874928693

# Ridge

In [25]:
from sklearn.linear_model import Ridge,Lasso

In [26]:
ridge = Ridge(alpha=0.5)

In [27]:
ridge.fit(X,y)

Ridge(alpha=0.5)

In [28]:
scores = cross_val_score(ridge, X, y, cv=3,scoring='neg_root_mean_squared_error')
np.mean(scores)

-1133.7059145708372

# Lasso

In [29]:
lasso = Lasso(alpha=1.0)

In [30]:
lasso.fit(X,y)

Lasso()

In [31]:
scores = cross_val_score(lasso, X, y, cv=3,scoring='neg_root_mean_squared_error')
np.mean(scores)

-1132.233629758277

# XgBoost

In [32]:
from xgboost import XGBRegressor

In [33]:
xgb = XGBRegressor()
xgb.fit(X,y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=12, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [34]:
scores = cross_val_score(xgb, X, y, cv=3,scoring='neg_root_mean_squared_error')
np.mean(scores)

-1179.7904326563078

# Submission

In [35]:
# pred_sales=ridge.predict(X_test)
# pred_sales=lasso.predict(X_test)
pred_sales=xgb.predict(X_test)

In [36]:
test.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NA,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NA,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NA,Tier 2,Supermarket Type1
4,FDY38,12.600,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [414]:
output = pd.DataFrame()
output['Item_Identifier'] = test['Item_Identifier']
output['Outlet_Identifier'] = test['Outlet_Identifier']
output['Item_Outlet_Sales'] = np.absolute(pred_sales)

In [428]:
sum(pred_sales<0)

43

In [416]:
output

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1832.855566
1,FDW14,OUT017,1483.475559
2,NCN55,OUT010,1932.457386
3,FDQ58,OUT017,2561.370096
4,FDY38,OUT027,5159.197899
...,...,...,...
5676,FDB58,OUT046,2296.953073
5677,FDD47,OUT018,2445.213515
5678,NCO17,OUT045,1827.336722
5679,FDJ26,OUT017,3548.552619


In [417]:
output.to_csv("Utkarsh.csv",index=False)